## 解线性方程组

### 三角方程组
解方程 $Ux = b$, 其中 $U$ 为非奇异上三角方阵.

$$\left[\begin{matrix}u_{11} & u_{12} & \dotsc & u_{1n} \\
\ &u_{22} &\dotsc & u_{2n}\\
\ &\ &\ddots & \vdots \\
\ & & &u_{nn}\end{matrix}\right]\left[\begin{matrix}x_1\\ x_2\\ \vdots \\ x_n\end{matrix}\right]
=\left[\begin{matrix}b_1\\ b_2\\ \vdots \\ b_n\end{matrix}\right].$$

显然可以先解 $x_n = \frac{b_n}{u_{nn}}$, 继而 $x_{n-1}=\frac{b_{n-1}-u_{n-1,n}x_{n}}{u_{n-1,n-1}}\dotsc$
$$x_k = \frac{b_k - \sum_{i=k+1}^n u_{ki}x_i}{u_{kk}}.$$

In [9]:
import numpy as np 

def SolveTriangular(U,b):
    n = U.shape[0]
    x = np.zeros(n)
    for k in range(n-1,-1,-1):
        x[k] = b[k]
        for i in range(k+1,n):
            x[k] -= U[k,i] * x[i]
        x[k] /= U[k,k]
    return x

式子 $$x_k = \frac{b_k - \sum_{i=k+1}^n u_{ki}x_i}{u_{kk}}$$

可以用向量-向量乘法改写, 即
$$x_k = \frac{b_k - U_{k,k+1:n}^Tx_{k+1:n}}{u_{kk}}.$$

如此调用1级BLAS比不调用可以加快运算速度.

In [10]:
def SolveTriangular2(U,b):
    n = U.shape[0]
    x = np.zeros(n)
    for k in range(n-1,-1,-1):
        x[k] = b[k]
        x[k] -= np.inner(U[k,k+1:],x[k+1:])
        x[k] /= U[k,k]
    return x

In [4]:
# 比较一下运行速度
# 生成一个上三角矩阵, 且对角元非零
n = 1000
#np.random.seed(1)
U = np.random.random((n,n)) + 1  # +1 使得对角元离零比较远
for i in range(n):
    U[i,:i] = 0
    
b = np.random.randn(n)

%timeit SolveTriangular(U.copy(),b.copy())
%timeit SolveTriangular2(U.copy(),b.copy())

979 ms ± 31.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
7.43 ms ± 1.06 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


## 高斯消元法

解方程 $Ax = b$, 其中 $A$ 是可逆方阵.

$$\left[\begin{matrix}a_{11} & a_{12} & \dotsc & a_{1n} \\
a_{21} &a_{22} &\dotsc & a_{2n}\\
\vdots &\vdots &\ddots & \vdots \\
a_{n1} &a_{n2} &\dotsc &a_{nn}\end{matrix}\right]\left[\begin{matrix}x_1\\ x_2\\ \vdots \\ x_n\end{matrix}\right]
=\left[\begin{matrix}b_1\\ b_2\\ \vdots \\ b_n\end{matrix}\right].$$

做初等行变换, 用方程组第 $i$ 行减去 $\frac{a_{i1}}{a_{11}}$ 倍的第一行.

$$a_i = a_i - \frac{a_{i1}}{a_{11}}a_1$$

$$b_i = b_i - \frac{a_{i1}}{a_{11}}b_1$$

变成

$$\left[\begin{matrix}a_{11} & a_{12} & \dotsc & a_{1n} \\
 0&a_{22}-\frac{a_{21}}{a_{11}}a_{12} &\dotsc & a_{2n}-\frac{a_{21}}{a_{11}}a_{1n}\\
\vdots &\vdots &\ddots & \vdots \\
0 &a_{n2}-\frac{a_{n1}}{a_{11}}a_{12} &\dotsc &a_{nn}-\frac{a_{n1}}{a_{11}}a_{1n}\end{matrix}\right]\left[\begin{matrix}x_1\\ x_2\\ \vdots \\ x_n\end{matrix}\right]
=\left[\begin{matrix}b_1\\ b_2-\frac{a_{21}}{a_{11}}b_{1}\\ \vdots \\ b_n-\frac{a_{n1}}{a_{11}}b_{n}\end{matrix}\right].$$

再做初等行变换, 用新的方程组第 $i\ (i> 2)$ 行减去 $\frac{a_{i2}'}{a_{22}'}$ 倍的第二行, ..., 直至方程组变为上三角方程组.

高斯消元法实际就是LU分解.

上述过程中, 第 $k$ 步是用目前方程组的第 $i\ (i> k)$ 行减去 $\frac{a_{ik}^{(k)}}{a_{kk}^{(k)}}$ 倍的第 $k$ 行.

即对于 $i>k+1$, 

$$a_i = a_i - \frac{a_{ik}}{a_{kk}}a_k$$

$$b_i = b_i - \frac{a_{ik}}{a_{kk}}b_k$$

可以写成矩阵-向量乘法

$$b_{k+1:n} = b_{k+1:n} - \frac{b_k}{a_{kk}}A_{k+1:n,k}$$

$$A_{k+1:n,k+1:n} = A_{k+1:n,k:n} - \frac{1}{a_{kk}}A_{k+1:n,k}A_{k,k+1:n}$$

$$A_{k+1:n,k} = 0.$$

In [5]:
def SolveLinear(A,b):
    n = A.shape[0]
    for k in range(n-1):
        b[k+1:] -= (b[k] / A[k,k]) * A[k+1:,k]
        A[k+1:,k+1:] -= (A[k+1:,k:k+1] / A[k,k]) @ A[k:k+1,k+1:]
        A[k+1:,k] = 0
    return SolveTriangular2(A,b)

n = 100
np.random.seed(1)
A = np.random.random((n,n)) - 0.5
b = np.random.randn(n)
sol = SolveLinear(A.copy(),b.copy())
print('Residual =',np.linalg.norm(A@sol - b))

Residual = 2.4363468245185613e-11


### 列选主元的高斯消元法 (Partial-Pivoting)

上述过程若中间执行到 $a_{kk}=0$ 则会中止. 或者, 若 $\frac{a_{ik}}{a_{kk}}$ 过大(上溢或在后续造成巨大绝对误差), 则数值稳定性弱, 即容易出现很大累计舍入误差. 对此可以进行“列选主元”:

在第 $k$ 步的时候: 对比 $A$ 的第 $k,\dotsc, n$ 行, 选取 $|a_{ik}|$ 最大的第 $ i$ 行与第 $k$ 行做行交换.

大多数情况下, 列选主元精度比不选主元的精度高. 也有例外如 Hilbert 矩阵.

In [6]:
def SolveLinear2(A,b):
    n = A.shape[0]
    for k in range(n-1):
        # 列选主元
        t = np.argmax(np.abs(A[k:,k])) + k
        tmp = A[t,k:].copy()
        A[t,k:] = A[k,k:]
        A[k,k:] = tmp
        
        b[k] , b[t] = b[t] , b[k]

        b[k+1:] -= (b[k] / A[k,k]) * A[k+1:,k]

        A[k+1:,k+1:] -= (A[k+1:,k:k+1] / A[k,k]) @ A[k:k+1,k+1:]
        A[k+1:,k] = 0
        
    return SolveTriangular2(A,b)

In [7]:
# 比较一下有无列选主元的精度
n = 1000
np.random.seed(0)
A = np.random.random((n,n)) - 0.5
realsol = np.random.randn(n)
b = A @ realsol
sol1 = SolveLinear(A.copy(),b.copy())
sol2 = SolveLinear2(A.copy(),b.copy())
print('Error without partial pivoting =',np.linalg.norm(realsol - sol1))
print('Error with    partial pivoting =',np.linalg.norm(realsol - sol2))
print('Residual without partial pivoting =',np.linalg.norm(A@sol1 - b))
print('Residual with    partial pivoting =',np.linalg.norm(A@sol2 - b))

Error without partial pivoting = 3.5521635042681607e-09
Error with    partial pivoting = 4.774189356382696e-12
Residual without partial pivoting = 2.054667425486347e-09
Residual with    partial pivoting = 3.3304376102372983e-12


### 增长因子 (Growth Factor)

若非零矩阵 $A$ 经过高斯消元法得到 $U$, 则记 $\rho = \frac{\max u_{ij}}{\max a_{ij}}$ 为增长因子.

**Theorem 1** $\rho \leqslant 2^{n-1}$.

**Theorem 2** 三对角矩阵的增长因子 $\rho \leqslant 2$. 

**Theorem 3** 对角占优矩阵的增长因子 $\rho \leqslant 2$.

### LU 分解

在高斯消元法中, 最后得到的上三角矩阵正好可以作为 LU 分解的 $U$. 而 $L$ 对角元均为 $1$ 的上三角矩阵. 且第 $k$ 步时, $$\frac{a_{ik}}{a_{kk}} = l_{ik}$$.

In [16]:
def LU(A):
    n = A.shape[0]
    L = np.eye(n)
    U = A.copy()
    for k in range(n-1):
        L[k+1:,k:k+1] = U[k+1:,k:k+1] / U[k,k]
        U[k+1:,k:] -= L[k+1:,k:k+1] @ U[k:k+1,k:]
        U[k+1:,k] = 0
    return L, U

n = 200
np.random.seed(0)
A = np.random.random((n,n)) - 0.5
L , U = LU(A)
# 比较一下 A 和 LU 有多接近
print('||A - LU|| =',np.linalg.norm(A - L @ U))

||A - LU|| = 4.848705487927859e-12


### LUP 分解

按照列主元的高斯消元法步骤中, 将所有置换提出来, 得到置换矩阵 $P$.
则 $PA = LU$. 一般来说 LUP 分解比 LU 分解数值稳定性更高.


In [15]:
def LUP(A):
    n = A.shape[0]
    L = np.eye(n)
    U = A.copy()
    permutation = list(range(n))
    for k in range(n-1):
        # 列选主元
        t = np.argmax(np.abs(U[k:,k])) + k
        tmp = U[t,k:].copy()
        U[t,k:] = U[k,k:]
        U[k,k:] = tmp
        
        tmp = L[t,:k].copy()
        L[t,:k] = L[k,:k]
        L[k,:k] = tmp

        permutation[k] , permutation[t] = permutation[t] , permutation[k]

        L[k+1:,k:k+1] = U[k+1:,k:k+1] / U[k,k]
        U[k+1:,k:] -= L[k+1:,k:k+1] @ U[k:k+1,k:]
        U[k+1:,k] = 0
    return L, U, permutation #np.eye(n)[permutation]

n = 200
np.random.seed(0)
A = np.random.random((n,n)) - 0.5
L , U , P = LUP(A)
# 比较一下 PA 和 LU 有多接近
print('||PA - LU|| =',np.linalg.norm(A[P] - L @ U))

||PA - LU|| = 8.101681572151669e-14
